In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

In [2]:
# Read the data
X = pd.read_csv('./data/housing_prices_competition/train.csv', index_col='Id')
X_test_full = pd.read_csv('./data/housing_prices_competition/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [3]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,False,True,False,False,False,False,False,False,False,True
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,False,False,False,True,False,False,False,False,True,False
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,False,False,False,True,False,False,False,False,True,False
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,False,False,False,True,False,False,False,False,True,False
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,False,False,False,True,False,False,False,False,True,False


In [4]:
model = XGBRegressor(random_state = 0)
model = model.fit(X_train, y_train)
preds = model.predict(X_valid)
error = mean_absolute_error(y_valid, preds)
print(f'{error=}')

error=18161.826171875


In [7]:
model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
model = model.fit(X_train, y_train, 
                  eval_set=[(X_valid, y_valid)],
                  verbose=False)

In [8]:
preds = model.predict(X_valid)
error = mean_absolute_error(y_valid, preds)
print(f'{error=}')

error=17224.28125
